DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
!pip install -q datasets sentencepiece rouge_score
!pip install -q transformers

DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
!pip -q install torch
import torch
print(torch.__version__)


DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
2.1.0+cu121


In [3]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
import gc
import os
from transformers import AutoTokenizer, LongT5ForConditionalGeneration
from datasets import load_dataset
import pandas as pd
import torch
from torch.cuda.amp import autocast

# Memory optimization for CUDA
max_split_size_mb = 256  # Set the max_split_size_mb value (e.g., 512 MB)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = f"max_split_size_mb:{max_split_size_mb}"
print(f"PYTORCH_CUDA_ALLOC_CONF is set to: {os.environ['PYTORCH_CUDA_ALLOC_CONF']}")
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

#when on Nvida machins 
# device = torch.device("cuda")
# print("device:", device)

#when you are on mac
device = torch.device("cpu")
print("device:", device)


# Load LongT5 Model and Tokenizer
# model_to_use = "google/long-t5-local-base"
model_to_use = "pszemraj/long-t5-tglobal-base-16384-book-summary"  # fined-tuned for summarization
longt5_model = LongT5ForConditionalGeneration.from_pretrained(model_to_use).to(device)
longt5_tokenizer = AutoTokenizer.from_pretrained(model_to_use)

# Load validation dataset from Hugging Face datasets
# dataset = load_dataset("allenai/mslr2022", "ms2", split='validation')
dataset = load_dataset("allenai/mslr2022", "ms2", split='train')


# Prepare DataFrame for output
# output_df = pd.DataFrame(columns=['ReviewID', 'Candidate_Summary', 'Target'])
output_df = pd.DataFrame(dataset)

print(dataset.column_names)

/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PYTORCH_CUDA_ALLOC_CONF is set to: max_split_size_mb:256
device: cpu
['review_id', 'pmid', 'title', 'abstract', 'target', 'background']


In [5]:
output_df.head(1)

,review_id,pmid,title,abstract,target,background
0,30760312,"[22776744, 25271670, 3493740, 1863023, 1629198...",[Improved Cell Survival and Paracrine Capacity...,[Although transplantation of adult bone marrow...,Conclusions SC therapy is effective for PAH in...,Background Despite significant progress in dru...


In [6]:
output_df.drop(['pmid', 'title','background'], axis = 1, inplace = True) 

In [7]:
output_df.head(1)

,review_id,abstract,target
0,30760312,[Although transplantation of adult bone marrow...,Conclusions SC therapy is effective for PAH in...


In [8]:
output_df['abstract'] = output_df['abstract'].apply(lambda x: "".join([f"Study : " + b for i,b in enumerate(x)]) )

In [9]:
output_df.head(3)

,review_id,abstract,target
0,30760312,Study : Although transplantation of adult bone...,Conclusions SC therapy is effective for PAH in...
1,19588356,Study : BACKGROUND Primary pulmonary hypertens...,There was a trend for endothelin receptor anta...
2,23893797,Study : BACKGROUND Although improved epicardia...,This present meta- analysis suggests that stat...


In [10]:
output_df.shape

(14188, 3)

In [11]:
# !pip install --upgrade transformers -q
# !pip uninstall transformers
# !pip install transformers
# !pip uninstall -y pytorch-lightning
# !pip install pytorch-lightning==1.5.10
# !pip install -q --upgrade simplet5

In [12]:
# !conda install protobuf

In [13]:
# from transformers import BertTokenizer, BertModel
# import torch
# import numpy as np
# from sklearn.cluster import KMeans
# from datasets import load_dataset
# import pandas as pd


# # Load the dataset and cut down to the first 5 for demonstration
# # dataset = load_dataset("allenai/mslr2022", "ms2", split='validation')
# # dataset = dataset.select(range(3))  # Use select to create a subset

# # Initialize BERT
# # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# # model = BertModel.from_pretrained('bert-base-uncased')

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# def bert_sentence_embeddings(sentences):
#     embeddings = []
#     for sentence in sentences:
#         inputs = tokenizer(sentence, return_tensors='pt', max_length=512, truncation=True)
#         with torch.no_grad():
#             outputs = model(**inputs)
#         embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
#     return np.array(embeddings)

# def select_top_sentences(sentences, embeddings, n_sentences=5):
#     if len(sentences) < n_sentences:
#         return ' '.join(sentences)
#     kmeans = KMeans(n_clusters=n_sentences, n_init=10)
#     kmeans.fit(embeddings)
#     top_sentences =[]
#     i = 0
#     while len(top_sentences) < n_sentences:
#         top_sentence_indices = np.argmin(
#         np.linalg.norm(embeddings[:, np.newaxis] - kmeans.cluster_centers_[i], axis=2), axis=0)
#         top_sentences.append(sentences[top_sentence_indices[0]])
#     return ' '.join(top_sentences)

# def process_row( abstract_text):
#     # Split abstract into sentences
#     sentences = abstract_text.split('. ')
#     # Generate embeddings for each sentence
#     embeddings = bert_sentence_embeddings(sentences)
#     # Select the top sentences from these embeddings
#     summary = select_top_sentences(sentences, embeddings)
        
#     return summary
        

                             
                             
# output_df['abstract'] = output_df['abstract'].apply(lambda x:process_row(x) )    

In [14]:

output_df.head(1)

,review_id,abstract,target
0,30760312,Study : Although transplantation of adult bone...,Conclusions SC therapy is effective for PAH in...


In [15]:
# import pandas as pd

# # Load the dataset from a CSV file
# dataset_path = '/mnt/wekamount/RI-Users/amir.moazami/Projects/266/266_final_proj/BioBERT_K_Means_extractive.csv'
# summaries_dataset = pd.read_csv(dataset_path , index_col=False)
# # Convert the dataset to a list of dictionaries
# data_list = summaries_dataset.to_dict(orient='records')
# summaries_dataset.head(1)

In [16]:
import pandas as pd

# Load the first dataset
dataset_path = '/mnt/wekamount/RI-Users/amir.moazami/Projects/266/266_final_proj/embeding_1.csv'
summaries_dataset = pd.read_csv(dataset_path, index_col=False)

# Rename the 'summary' column to 'abstract'
summaries_dataset.rename(columns={'summary': 'abstract'}, inplace=True)

# Convert the 'review_id' column in summaries_dataset to integer (if it's not already)
summaries_dataset['review_id'] = summaries_dataset['review_id'].astype(int)

# Load the second dataset
# (Assuming you have already loaded this dataset as 'output_df' in your previous steps)

# Convert the 'review_id' column in output_df to integer (if it's not already)
output_df['review_id'] = output_df['review_id'].astype(int)

# Merge the two datasets based on 'review_id'
merged_dataset = pd.merge(output_df, summaries_dataset[['review_id', 'abstract']], on='review_id', how='left')
# merged_dataset.head(2)
# merged_dataset.shape[0]
# Replace the 'abstract' column in output_df with the one from summaries_dataset
output_df['abstract'] = merged_dataset['abstract_y']
output_df.head(2)

# Now output_df will have the updated abstracts from summaries_dataset

,review_id,abstract,target
0,30760312,"At 1 week posttransplantation , the number of ...",Conclusions SC therapy is effective for PAH in...
1,19588356,The changes in mean pulmonary-artery pressure ...,There was a trend for endothelin receptor anta...


In [17]:
output_df.rename(columns={"target":"target_text", "abstract":"source_text"}, inplace = True)


In [18]:
output_df.head(1)

,review_id,source_text,target_text
0,30760312,"At 1 week posttransplantation , the number of ...",Conclusions SC therapy is effective for PAH in...


In [19]:
output_df.shape

(14188, 3)

In [20]:
# Load LongT5 Model and Tokenizer
# model_to_use = "google/long-t5-local-base"
# model_to_use = "pszemraj/long-t5-tglobal-base-16384-book-summary"  # fined-tuned for summarization
# longt5_model = LongT5ForConditionalGeneration.from_pretrained(model_to_use).to(device)
# longt5_tokenizer = AutoTokenizer.from_pretrained(model_to_use)
from simplet5 import SimpleT5
model=SimpleT5()
model.from_pretrained(model_type="longt5",model_name="pszemraj/long-t5-tglobal-base-16384-book-summary")

2023-12-01 16:34:11.937221: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-01 16:34:12.158546: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Global seed set to 42


In [ ]:
# model.train(train_df=output_df,
#             eval_df=output_df, 
#             source_max_token_len=2048, 
#             target_max_token_len=1024, 
#             batch_size=2, max_epochs=5, use_gpu=True)
# Split the data into training and evaluation sets
import pandas as pd
from sklearn.model_selection import train_test_split
from simplet5 import SimpleT5
# Initialize SimpleT5
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="pszemraj/long-t5-tglobal-base-16384-book-summary")

train_df, eval_df = train_test_split(output_df, test_size=0.2)
model.train(train_df=train_df,
            eval_df=eval_df, 
            source_max_token_len=2048, 
            target_max_token_len=1024, 
            batch_size=2, 
            max_epochs=8, 
            use_gpu=True) 

You are using a model of type longt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at pszemraj/long-t5-tglobal-base-16384-book-summary and are newly initialized: ['encoder.block.6.layer.0.SelfAttention.o.weight', 'encoder.block.2.layer.0.SelfAttention.k.weight', 'encoder.block.7.layer.0.SelfAttention.k.weight', 'encoder.block.9.layer.0.SelfAttention.o.weight', 'encoder.block.3.layer.0.SelfAttention.k.weight', 'encoder.block.8.layer.0.SelfAttention.o.weight', 'encoder.block.11.layer.0.SelfAttention.q.weight', 'encoder.block.6.layer.0.SelfAttention.q.weight', 'encoder.block.2.layer.0.SelfAttention.o.weight', 'encoder.block.4.layer.0.SelfAttention.k.weight', 'encoder.block.7.layer.0.SelfAttention.v.weight', 'encoder.block.8.layer.0.SelfAttention.v.weight', 'encoder.block.3.layer.0.SelfAttention.q.weight', 'encoder.block.8.layer.0.SelfA

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Global seed set to 42
/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  80%|███████▉  | 5675/7094 [1:51:24<27:51,  1.18s/it, loss=2.89, v_num=260027, train_loss_step=2.720]  
Validating: 0it [00:00, ?it/s]
Epoch 1:  11%|█         | 790/7094 [15:30<2:03:41,  1.18s/it, loss=2.8, v_num=260027, train_loss_step=3.020, val_loss_step=2.710, val_loss_epoch=2.650, train_loss_epoch=3.160]  

...

Epoch 1:  80%|███████▉  | 5675/7094 [1:51:10<27:48,  1.18s/it, loss=2.7, v_num=260027, train_loss_step=2.570, val_loss_step=2.710, val_loss_epoch=2.650, train_loss_epoch=3.160]   
Validating: 0it [00:00, ?it/s]
Epoch 2:  59%|█████▊    | 4165/7094 [1:21:27<57:16,  1.17s/it, loss=2.65, v_num=260027, train_loss_step=2.880, val_loss_step=2.600, val_loss_epoch=2.570, train_loss_epoch=2.780]  

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 3:  80%|███████▉  | 5675/7094 [1:50:48<27:42,  1.17s/it, loss=2.53, v_num=260027, train_loss_step=2.580, val_loss_step=2.650, val_loss_epoch=2.550, train_loss_epoch=2.610]  
Validating: 0it [00:00, ?it/s]
Epoch 3:  98%|█████████▊| 6955/7094 [2:00:13<02:24,  1.04s/it, loss=2.53, v_num=260027, train_loss_step=2.580, val_loss_step=2.650, val_loss_epoch=2.550, train_loss_epoch=2.610]


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 6:  52%|█████▏    | 3705/7094 [1:12:18<1:06:08,  1.17s/it, loss=2.1, v_num=260027, train_loss_step=2.320, val_loss_step=2.550, val_loss_epoch=2.540, train_loss_epoch=2.280] 

In [ ]:
output_df.head(1)

# infrence


In [1]:
import gc
import os
from transformers import AutoTokenizer, LongT5ForConditionalGeneration
from datasets import load_dataset
import pandas as pd
import torch
from torch.cuda.amp import autocast

# Memory optimization for CUDA
max_split_size_mb = 256  # Set the max_split_size_mb value (e.g., 512 MB)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = f"max_split_size_mb:{max_split_size_mb}"
print(f"PYTORCH_CUDA_ALLOC_CONF is set to: {os.environ['PYTORCH_CUDA_ALLOC_CONF']}")
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

#when on Nvida machins 
device = torch.device("cuda")
print("device:", device)

#when you are on mac
device = torch.device("cpu")
print("device:", device)

# Load validation dataset from Hugging Face datasets
dataset = load_dataset("allenai/mslr2022", "ms2", split='validation')
# dataset = load_dataset("allenai/mslr2022", "ms2", split='train')


# Prepare DataFrame for output
# output_df = pd.DataFrame(columns=['ReviewID', 'Candidate_Summary', 'Target'])
output_df = pd.DataFrame(dataset)

print(dataset.column_names)

/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PYTORCH_CUDA_ALLOC_CONF is set to: max_split_size_mb:256
device: cuda
device: cpu
['review_id', 'pmid', 'title', 'abstract', 'target', 'background']


In [2]:
output_df.drop(['pmid', 'title','background'], axis = 1, inplace = True) 
output_df['abstract'] = output_df['abstract'].apply(lambda x: "".join([f"Study : " + b for i,b in enumerate(x)]) )
output_df.head(3)
output_df.shape

(2021, 3)

In [3]:
output_df.head(3)

,review_id,abstract,target
0,28514886,Study : ABSTRACT A healthy intestinal microbio...,Current evidence from systematic review and me...
1,18842808,Study : The effects of the soluble fiber konja...,The use of glucomannan did not appear to signi...
2,24297836,Study : The aims of this study were 1 ) to eva...,Ensuring that the characteristics of the histo...


# in case you need Kmeans inputsimport pandas as pd


In [4]:

import pandas as pd

# Load the first dataset
dataset_path = '/mnt/wekamount/RI-Users/amir.moazami/Projects/266/266_final_proj/BioBERT_K_Means_extractive.csv'
summaries_dataset = pd.read_csv(dataset_path, index_col=0)

# Rename the 'summary' column to 'abstract'
summaries_dataset.rename(columns={'summary': 'abstract'}, inplace=True)

# Convert the 'review_id' column in summaries_dataset to integer (if it's not already)
summaries_dataset['review_id'] = summaries_dataset['review_id'].astype(int)

summaries_dataset.shape



(2021, 2)

In [5]:
# Load the second dataset
# (Assuming you have already loaded this dataset as 'output_df' in your previous steps)

# Convert the 'review_id' column in output_df to integer (if it's not already)
output_df['review_id'] = output_df['review_id'].astype(int)

# Merge the two datasets based on 'review_id'
merged_dataset = pd.merge(output_df, summaries_dataset[['review_id', 'abstract']], on='review_id', how='left')
# merged_dataset.head(2)
# merged_dataset.shape[0]
# Replace the 'abstract' column in output_df with the one from summaries_dataset
output_df['abstract'] = merged_dataset['abstract_y']
output_df.head(2)

# Now output_df will have the updated abstracts from summaries_dataset

,review_id,abstract,target
0,28514886,Breast-fed infants typically have an intestina...,Current evidence from systematic review and me...
1,18842808,No adverse effects were observed . The effects...,The use of glucomannan did not appear to signi...


In [6]:
output_df.shape

(2021, 3)

In [7]:
output_df.head(3)

,review_id,abstract,target
0,28514886,Breast-fed infants typically have an intestina...,Current evidence from systematic review and me...
1,18842808,No adverse effects were observed . The effects...,The use of glucomannan did not appear to signi...
2,24297836,Autonomic cardiovascular dysfunction accompani...,Ensuring that the characteristics of the histo...


In [8]:
output_df.rename(columns={"target":"target_text", "abstract":"source_text"}, inplace = True)


In [13]:
# model.predict(output_df['source_text'][0])
from simplet5 import SimpleT5

# # Load your trained model
model = SimpleT5()
model.load_model("t5","outputs/training_On_full_train(lowerModelsOnValidationKmeans)/simplet5-epoch-8-train-loss-2.0239-val-loss-2.6407", use_gpu=True)

text_to_summarize=output_df['source_text'][0]
model.predict(text_to_summarize)

['Conclusions The results of this systematic review and meta- analysis support the hypothesis that a high intake of galifidolide is associated with reduced risk of developing P. fecalis.']

In [14]:
output_df.shape

(2021, 3)

In [15]:
output_df.head(1)

,review_id,source_text,target_text
0,28514886,Breast-fed infants typically have an intestina...,Current evidence from systematic review and me...


In [16]:
# import gc
# import os
# from transformers import AutoTokenizer, LongT5ForConditionalGeneration
# from datasets import load_dataset
# import pandas as pd
# import torch
# from torch.cuda.amp import autocast

# # Memory optimization for CUDA
# max_split_size_mb = 256  # Set the max_split_size_mb value (e.g., 512 MB)
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = f"max_split_size_mb:{max_split_size_mb}"
# print(f"PYTORCH_CUDA_ALLOC_CONF is set to: {os.environ['PYTORCH_CUDA_ALLOC_CONF']}")
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# # when on Nvida machins 
# device = torch.device("cuda")
# print("device:", device)

# # #when you are on mac
# # device = torch.device("cpu")
# # print("device:", device)


# # Load LongT5 Model and Tokenizer
# # model_to_use = "google/long-t5-local-base"
# # model_to_use = "pszemraj/long-t5-tglobal-base-16384-book-summary"  # fined-tuned for summarization
# # longt5_model = LongT5ForConditionalGeneration.from_pretrained(model_to_use).to(device)
# # longt5_tokenizer = AutoTokenizer.from_pretrained(model_to_use)

# # Load validation dataset from Hugging Face datasets
# # dataset = load_dataset("allenai/mslr2022", "ms2", split='validation')
# dataset = load_dataset("allenai/mslr2022", "ms2", split='validation')


# # Prepare DataFrame for output
# # output_df = pd.DataFrame(columns=['ReviewID', 'Candidate_Summary', 'Target'])
# output_df = pd.DataFrame(dataset)

# print(dataset.column_names)

In [17]:

# output_df.drop(['pmid', 'title','background'], axis = 1, inplace = True) 
# output_df['abstract'] = output_df['abstract'].apply(lambda x: "".join([f"Study : " + b for i,b in enumerate(x)]) )
# output_df.rename(columns={"target":"target_text", "abstract":"source_text"}, inplace = True)


In [18]:
output_df.shape

(2021, 3)

In [19]:
output_df.head(1)

,review_id,source_text,target_text
0,28514886,Breast-fed infants typically have an intestina...,Current evidence from systematic review and me...


In [20]:
# import pandas as pd
# from simplet5 import SimpleT5

# # Load your trained model
# model = SimpleT5()
# model.load_model("t5", "outputs/simplet5-epoch-9-train-loss-1.9464-val-loss-2.6663", use_gpu=True)

# # Assuming 'output_df' is already loaded and has columns 'review_id', 'source_text', and 'target_text'
# batch_size = 1

# # Initialize an empty DataFrame for the summaries
# summary_df = pd.DataFrame()

# # Process data in batches and generate summaries
# for i in range(0, len(output_df), batch_size):
#     batch_abstracts = output_df['source_text'][i: i + batch_size]
#     batch_review_ids = output_df['review_id'][i: i + batch_size]
#     batch_targets = output_df['target_text'][i: i + batch_size] if 'target_text' in output_df.columns else [''] * batch_size

#     # Generate summaries
#     batch_summaries = [model.predict(abstract) for abstract in batch_abstracts]

#     # Create a temporary DataFrame and append it to the summary DataFrame
#     temp_df = pd.DataFrame({
#         'ReviewID': batch_review_ids,
#         'Candidate_Summary': batch_summaries,
#         'Target': batch_targets
#     })
#     summary_df = pd.concat([summary_df, temp_df], ignore_index=True)

# # Save the summary DataFrame to a CSV file
# summary_df.to_csv('model_evaluation_output.csv', index=False)

# # Display the summary DataFrame (optional)
# display(summary_df)

# # Examine a single review example (optional)
# review_row = 5
# print("CANDIDATE")
# print(summary_df.loc[review_row, "Candidate_Summary"])
# print("TARGET")
# print(summary_df.loc[review_row, "Target"])




In [21]:

import pandas as pd
from simplet5 import SimpleT5
import torch
import gc

# Load your trained model
model = SimpleT5()
# model.load_model("t5", "outputs/simplet5-epoch-7-train-loss-2.1058-val-loss-2.5727", use_gpu=True)
model.load_model("t5","outputs/training_On_full_train(lowerModelsOnValidationKmeans)/simplet5-epoch-8-train-loss-2.0239-val-loss-2.6407", use_gpu=True)


# Assuming 'output_df' is already loaded
batch_size = 1
summary_df = pd.DataFrame()

for i in range(0, len(output_df), batch_size):
    # Truncate source text to a reasonable length
    batch_abstracts = output_df['source_text'][i: i + batch_size].str[:512]  # Adjust as needed
    batch_review_ids = output_df['review_id'][i: i + batch_size]
    batch_targets = output_df['target_text'][i: i + batch_size] if 'target_text' in output_df.columns else [''] * batch_size

    # Generate summaries with optimized parameters
    batch_summaries = [model.predict(abstract) for abstract in batch_abstracts]

    temp_df = pd.DataFrame({
        'ReviewID': batch_review_ids,
        'Candidate_Summary': batch_summaries,
        'Target': batch_targets
    })
    summary_df = pd.concat([summary_df, temp_df], ignore_index=True)

    # Memory cleanup
    gc.collect()
    torch.cuda.empty_cache()

summary_df.to_csv('Regular_fineTuend_Kmeans_input_validation.csv', index=False)
display(summary_df)


/mnt/wekamount/RI-Users/amir.moazami/Projects/266/.venv/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


,ReviewID,Candidate_Summary,Target
0,28514886,"[B-fibrinol, B. ifidium and B. boulardiium are...",Current evidence from systematic review and me...
1,18842808,[Conclusions GM-C6 concentrations are signific...,The use of glucomannan did not appear to signi...
2,24297836,[Autonomic systolic BP is significantly higher...,Ensuring that the characteristics of the histo...
3,32367221,[Conclusions : Single-port ACL reconstruction ...,The QT autograft detected comparable rate of L...
4,25038833,[The results of the meta-analyses indicate tha...,medicines with anti-cholinergic properties hav...
...,...,...,...
2016,19776504,[There was no evidence of a difference between...,This systematic review with meta- analysis fou...
2017,27505198,[Weight loss is associated with significant re...,A wide range of techniques have been evaluated...
2018,25251296,[The BMD of low- and high-density lipoprotein ...,"First , during anorexia nervosa adolescent fem..."
2019,23235652,[The results of this meta- analysis support th...,There is no convincing evidence that zinc supp...


In [22]:
print ("hello")

hello


In [23]:
                       
        
#         # Combine the summaries from each abstract
#         combined_summary += summary + ' '

#     return {"review_id": review_id, "summary": combined_summary.strip()}

# Apply the function to each element of the dataset
# summaries_dataset = dataset.map(process_row)

# # Convert to pandas DataFrame
# df = pd.DataFrame(summaries_dataset)
# df = df[['review_id', 'summary']]
# # Save to CSV
# csv_file_path = 'test.csv'  # Update with your desired file path
# df.to_csv(csv_file_path, index=True)

# print(f"Saved summaries to {csv_file_path}")